<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Случайны-лес" data-toc-modified-id="Случайны-лес-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Случайны лес</a></span></li><li><span><a href="#Тестирование-лучше-модели" data-toc-modified-id="Тестирование-лучше-модели-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Тестирование лучше модели</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
from IPython.core.display import display, HTML
display(HTML('<style>.container { width:90% !important; }</style>'))

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stopwords = stopwords.words('english')

pd.set_option('display.max_colwidth', 1000)
import re


from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import f1_score
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## Подготовка

In [3]:
comments = pd.read_csv('/datasets/toxic_comments.csv')

In [4]:
comments.head(10)

,Unnamed: 0,text,toxic
0,0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,4,"You, sir, are my hero. Any chance you remember what page that's on?",0
5,5,"""\n\nCongratulations from me as well, use the tools well. · talk """,0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,"Your vandalism to the Matt Shirvington article has been reverted. Please don't do it again, or you will be banned.",0
8,8,"Sorry if the word 'nonsense' was offensive to you. Anyway, I'm not intending to write anything in the article(wow they would jump on me for vandalism), I'm merely requesting that it be more encyclopedic so one can use it for school as a reference. I have been to the selective breeding page but it's almost a stub. It points to 'animal breeding' which is a short messy article that gives you no info. There must be someone around with expertise in eugenics? 93.161.107.169",0
9,9,alignment on this subject and which are contrary to those of DuLithgow,0


In [5]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [6]:
print(f'Количество дубликатов: {comments.duplicated().sum()}')

Количество дубликатов: 0


В данных есть столбец Unnamed: 0 - похоже это нумерация. удалим его

In [7]:
comments = comments.drop('Unnamed: 0', axis=1)

In [8]:
comments.head(10)

,text,toxic
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,"You, sir, are my hero. Any chance you remember what page that's on?",0
5,"""\n\nCongratulations from me as well, use the tools well. · talk """,0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,"Your vandalism to the Matt Shirvington article has been reverted. Please don't do it again, or you will be banned.",0
8,"Sorry if the word 'nonsense' was offensive to you. Anyway, I'm not intending to write anything in the article(wow they would jump on me for vandalism), I'm merely requesting that it be more encyclopedic so one can use it for school as a reference. I have been to the selective breeding page but it's almost a stub. It points to 'animal breeding' which is a short messy article that gives you no info. There must be someone around with expertise in eugenics? 93.161.107.169",0
9,alignment on this subject and which are contrary to those of DuLithgow,0


В данных присутствуют разделители строки, заглавные символы. Очистим и произведем лемматизацию

In [9]:
%%time
m = WordNetLemmatizer()

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def clear_lemmatize(text):
    text = text.lower()
    lemm_text = " ".join(m.lemmatize(w ,get_wordnet_pos(w)) for w in nltk.word_tokenize(text))
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
    return " ".join(cleared_text.split())

comments['lemm_text'] = comments['text'].apply(clear_lemmatize)

del m

CPU times: user 24min 4s, sys: 2min 12s, total: 26min 17s
Wall time: 26min 32s


In [10]:
display(comments.head(10))

,text,toxic,lemm_text
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,explanation why the edits make under my username hardcore metallica fan be revert they be n t vandalism just closure on some gas after i vote at new york doll fac and please do n t remove the template from the talk page since i m retire now
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,d aww he match this background colour i m seemingly stuck with thanks talk january utc
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,hey man i m really not try to edit war it s just that this guy be constantly remove relevant information and talk to me through edits instead of my talk page he seem to care more about the format than the actual info
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0,more i ca n t make any real suggestion on improvement i wonder if the section statistic should be later on or a subsection of type of accident i think the reference may need tidy so that they be all in the exact same format ie date format etc i can do that later on if no one else do first if you have any preference for format style on reference or want to do it yourself please let me know there appear to be a backlog on article for review so i guess there may be a delay until a reviewer turn up it s list in the relevant form eg wikipedia good article nominations transport
4,"You, sir, are my hero. Any chance you remember what page that's on?",0,you sir be my hero any chance you remember what page that s on
5,"""\n\nCongratulations from me as well, use the tools well. · talk """,0,congratulation from me a well use the tool well talk
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,cocksucker before you piss around on my work
7,"Your vandalism to the Matt Shirvington article has been reverted. Please don't do it again, or you will be banned.",0,your vandalism to the matt shirvington article have be revert please do n t do it again or you will be ban
8,"Sorry if the word 'nonsense' was offensive to you. Anyway, I'm not intending to write anything in the article(wow they would jump on me for vandalism), I'm merely requesting that it be more encyclopedic so one can use it for school as a reference. I have been to the selective breeding page but it's almost a stub. It points to 'animal breeding' which is a short messy article that gives you no info. There must be someone around with expertise in eugenics? 93.161.107.169",0,sorry if the word nonsense be offensive to you anyway i m not intend to write anything in the article wow they would jump on me for vandalism i m merely request that it be more encyclopedic so one can use it for school a a reference i have be to the selective breeding page but it s almost a stub it point to animal breeding which be a short messy article that give you no info there must be someone around with expertise in eugenics
9,alignment on this subject and which are contrary to those of DuLithgow,0,alignment on

Удалим исходные комментарии после лематизации они нам не понадобятся

In [11]:
comments = comments.drop(['text'], axis=1)

## Обучение

Подготовим данные

In [12]:
features = comments.drop(['toxic'], axis=1)
target = comments['toxic']

Разобьем выборку по отношению 60/20/20. Уменьшим количество кроссвалидаций до 3 из-за размера выборки.

In [13]:
features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              test_size=0.4, 
                                                                              random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                            target_valid, 
                                                                            test_size=0.5,
                                                                            random_state=12345)


In [14]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

features_train = count_tf_idf.fit_transform(features_train['lemm_text'])
features_valid = count_tf_idf.transform(features_valid['lemm_text'])
features_test = count_tf_idf.transform(features_test['lemm_text'])
print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)
cv_counts = 3

(95575, 114770)
(31858, 114770)
(31859, 114770)


### Логистическая регрессия

In [15]:
pipe = Pipeline([
    (
    ('model', LogisticRegression(random_state=12345, solver='liblinear', max_iter=200))
    )
    ])
param_grid = [
        {

            'model': [LogisticRegression(random_state=12345, solver='liblinear')],
            'model__penalty' : ['l1', 'l2'],
            'model__C': list(range(1,15,3))
        }]

In [16]:
lr_grid = GridSearchCV(pipe, param_grid=param_grid, scoring='f1', cv=cv_counts, verbose=True, n_jobs=-1)
best_grid = lr_grid.fit(features_train, target_train)
print('Best parameters is:', lr_grid.best_params_)
print('Best score is:', lr_grid.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best parameters is: {'model': LogisticRegression(C=4, penalty='l1', random_state=12345, solver='liblinear'), 'model__C': 4, 'model__penalty': 'l1'}
Best score is: 0.7616208246030859


Модель логистической регресси на тренировочной выборке показала F1 0.7616208246030859

### Случайны лес

In [17]:
params_forest = {
    'n_estimators': list(range(50,300,50)),
    'max_depth':[5,15]
}
rfc = RandomForestClassifier(random_state=12345)
                                 
grid = GridSearchCV(rfc, param_grid=params_forest, scoring='f1', cv=cv_counts, verbose=True, n_jobs=-1)
best_grid = grid.fit(features_train, target_train)
print('Best parameters is:', grid.best_params_)
print('Best score is:', grid.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best parameters is: {'max_depth': 15, 'n_estimators': 50}
Best score is: 0.00041433602651750565


 Случайный лес с ппараметрами 'max_depth': 15, 'n_estimators': 50 показал результат 0.00041433602651750565

### Тестирование лучше модели

проверим Логистическую регрессию на валидационной выборке

In [18]:
lr = LogisticRegression(random_state=12345, C = 4, penalty = 'l1', solver='liblinear', max_iter=200)
lr.fit(features_train, target_train)
valid_pred = lr.predict(features_valid)
print('F1',f1_score(target_valid, valid_pred))

F1 0.781962338949455


In [19]:
test_pred = lr.predict(features_test)
print(metrics.classification_report(target_test, test_pred), metrics.f1_score(target_test, test_pred))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98     28639
           1       0.85      0.70      0.77      3220

    accuracy                           0.96     31859
   macro avg       0.91      0.84      0.87     31859
weighted avg       0.96      0.96      0.96     31859
 0.7687051588657329


На тестовой выборке модель Логистической ргегресси показала F1: 0.7687051588657329

## Выводы

В данном проекте мы провели подготовку данных для обучения моделец. Произвели лематизацию, сентимент анализ. Далее раздели данные на тренировочную, валидацилнную и тестовую выборки.
Произвели кросс-валидацию Логистической регресси и случаного леса. Во время кроссавалидации лучший результат показала логистическая леегрессия f1:  0.7687051588657329
    
На тестовой выборке Наша модель показалал 0.7710925515595705 что уклдаывается в тербования задачи.